In [1]:
# Import required packages (will fail if not available)
import xarray as xr
import pysnptools

print(f"✓ xarray {xr.__version__}")
print(f"✓ pysnptools {type(pysnptools)}")

✓ xarray 2025.7.1
✓ pysnptools <class 'module'>


In [2]:
# Get sample BED file using PySnpTools
from pysnptools.snpreader import Bed
from pysnptools.util import example_file

bed_file = example_file("tests/datasets/all_chr.maf0.001.N300.*")
snp_reader = Bed(bed_file, count_A1=True)
print(f"Shape: {snp_reader.shape} (individuals × SNPs)")

Shape: (300, 1015) (individuals × SNPs)


In [9]:
# Convert PySnpTools BED data to xarray Dataset with full genomic metadata
import numpy as np

# Create xarray Dataset using numeric indexes (avoids MultiIndex complexity and ensures Zarr compatibility)
xarray_form = xr.Dataset(
    {
        "genotypes": (["individual", "snp"], snp_reader.read().val)
    },
    coords={
        "individual": range(snp_reader.iid_count),
        "fid": (["individual"], [fid for fid, _iid in snp_reader.read().iid]),  # Family IDs
        "iid": (["individual"], [iid for _fid, iid in snp_reader.read().iid]),  # Individual IDs
        
        "snp": range(snp_reader.sid_count),
        "sid": (["snp"], snp_reader.sid),                                                 # SNP IDs
        "chromosome": (["snp"], np.nan_to_num(snp_reader.pos[:, 0], nan=0).astype(int)),  # Chromosome (NaN→0)
        "cm_position": (["snp"], snp_reader.pos[:, 1]),                                   # Genetic position
        "bp_position": (["snp"], np.nan_to_num(snp_reader.pos[:, 2], nan=0).astype(int)), # Physical position (NaN→0)
    },
    attrs={
        "description": "Genotype data from PySnpTools BED file",
        "encoding": "0=homozygous ref, 1=heterozygous, 2=homozygous alt, NaN=missing",
        "source": bed_file,
    }
)

display(xarray_form)

<xarray.Dataset> Size: 3MB
Dimensions:      (individual: 300, snp: 1015)
Coordinates:
  * individual   (individual) int64 2kB 0 1 2 3 4 5 ... 294 295 296 297 298 299
    fid          (individual) <U4 5kB 'POP1' 'POP1' 'POP1' ... 'POP1' 'POP1'
    iid          (individual) <U4 5kB '0' '12' '44' ... '2987' '2998' '2999'
  * snp          (snp) int64 8kB 0 1 2 3 4 5 6 ... 1009 1010 1011 1012 1013 1014
    sid          (snp) <U10 41kB '1_12' '1_34' '1_10' ... '23_49' '23_2' '23_3'
    chromosome   (snp) int64 8kB 1 1 1 1 1 1 1 1 1 ... 23 23 23 23 23 23 23 23
    cm_position  (snp) float64 8kB 0.008008 0.02302 0.07007 ... 0.9479 0.995
    bp_position  (snp) int64 8kB 0 1 4 6 7 8 9 10 11 ... 44 46 47 48 49 50 51 52
Data variables:
    genotypes    (individual, snp) float64 2MB 0.0 0.0 1.0 0.0 ... 0.0 2.0 0.0
Attributes:
    description:  Genotype data from PySnpTools BED file
    encoding:     0=homozygous ref, 1=heterozygous, 2=homozygous alt, NaN=mis...
    source:       C:\Users\carlk\AppData\Local\Temp/hashdown/072f91aaa72b5340...

In [10]:
# Import zarr explicitly and save to Zarr format
import zarr
print(f"✓ zarr {zarr.__version__}")

# Save to Zarr format for efficient storage and access
zarr_path = "all_chr.maf0.001.N300.zarr"
xarray_form.to_zarr(zarr_path, mode='w')

# Verify by loading back lazily
zarr_form = xr.open_zarr(zarr_path)
zarr_form


✓ zarr 3.0.10


o:\Projects\Science\snpxarray\.venv\Lib\site-packages\zarr\codecs\vlen_utf8.py:44: UserWarning: The codec `vlen-utf8` is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  return cls(**configuration_parsed)
o:\Projects\Science\snpxarray\.venv\Lib\site-packages\zarr\core\array.py:4275: UserWarning: The dtype `<U4` is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  meta = AsyncArray._create_metadata_v3(
o:\Projects\Science\snpxarray\.venv\Lib\site-packages\zarr\codecs\vlen_utf8.py:44: UserWarning: The codec `vlen-utf8` is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  return cls(**configuration_parsed)
o:\Projects\Science\snpxarray\.venv\Lib\site-packages\zarr\core\array.py:4275: UserWarning: The dtype `<U4` is curr

<xarray.Dataset> Size: 2MB
Dimensions:      (individual: 300, snp: 1015)
Coordinates:
    chromosome   (snp) int64 8kB ...
    fid          (individual) object 2kB ...
    bp_position  (snp) int64 8kB ...
    cm_position  (snp) float64 8kB ...
  * individual   (individual) int64 2kB 0 1 2 3 4 5 ... 294 295 296 297 298 299
    sid          (snp) object 8kB ...
  * snp          (snp) int64 8kB 0 1 2 3 4 5 6 ... 1009 1010 1011 1012 1013 1014
    iid          (individual) object 2kB ...
Data variables:
    genotypes    (individual, snp) float64 2MB ...
Attributes:
    description:  Genotype data from PySnpTools BED file
    encoding:     0=homozygous ref, 1=heterozygous, 2=homozygous alt, NaN=mis...
    source:       C:\Users\carlk\AppData\Local\Temp/hashdown/072f91aaa72b5340...

In [ ]:
print(f"Dataset loaded with data variables: {list(lazy_genotypes.data_vars.keys())}")
print(f"Dataset shape: {lazy_genotypes.sizes}")
print(f"Coordinates preserved: {list(lazy_genotypes.coords.keys())}")
print(f"Attributes preserved: {bool(lazy_genotypes.attrs)}")

# Show that it works the same way
print(f"\nFirst individual from disk: ({lazy_genotypes.fid.values[0]}, {lazy_genotypes.iid.values[0]})")
print(f"First SNP from disk: {lazy_genotypes.sid.values[0]}")
print(f"First genotype from disk: {lazy_genotypes.genotypes.values[0, 0]}")

# Show efficient filtering examples
print(f"\nFiltering examples:")
print("# By chromosome: lazy_genotypes.where(lazy_genotypes.chromosome == 1, drop=True)")
print("# By family: lazy_genotypes.where(lazy_genotypes.fid == 'POP1', drop=True)")
print("# By SNP ID: lazy_genotypes.where(lazy_genotypes.sid.isin(['1_12', '1_34']), drop=True)")